In [1]:
import requests
import bs4
from bs4 import BeautifulSoup
from urllib.parse import urljoin
import pandas as pd

In [85]:
url_inicial = 'http://www.revistas.unam.mx/index.php/rmcpys/issue/archive?issuesPage=7#issues'
url_root = 'http://www.revistas.unam.mx/index.php/rmcpys/issue/archive?issuesPage=7#issues'
r=requests.get(url_inicial)

In [86]:
soup = BeautifulSoup(r.text, 'html.parser')

In [87]:
box = soup.find('div', id='issues')
volumen=box.findAll('div', style='float: left; width: 100%;')
vol = [x.find('a').get('href')for x in volumen]
vol=[urljoin(url_root,i) for i in vol]
vol

['http://www.revistas.unam.mx/index.php/rmcpys/issue/view/5924',
 'http://www.revistas.unam.mx/index.php/rmcpys/issue/view/5955',
 'http://www.revistas.unam.mx/index.php/rmcpys/issue/view/5997',
 'http://www.revistas.unam.mx/index.php/rmcpys/issue/view/6008',
 'http://www.revistas.unam.mx/index.php/rmcpys/issue/view/6037']

In [88]:
def get_url_items(sopa, url): 
    box = soup.find('div', id='issues')
    volumen=box.findAll('div', style='float: left; width: 100%;')
    vol = [x.find('a').get('href')for x in volumen]
    vol=[urljoin(url_root,i) for i in vol]
    return vol

In [89]:
links_items=[]
i=0
while i<1:
    i+=1
    print(f'Estoy en la pagina {url_inicial}')
    r_pag=requests.get(url_inicial)
    s_p=BeautifulSoup(r_pag.text,'html.parser')
    links=get_url_items(s_p, url_inicial)
    links_items.append(links) 

Estoy en la pagina http://www.revistas.unam.mx/index.php/rmcpys/issue/archive?issuesPage=7#issues


In [90]:
list_scraper=[]
for i in links_items:
    for j in i:
        list_scraper.append(j)
len(list_scraper)

5

In [91]:
uno=list_scraper[0]
r_item=requests.get(uno)
s_item=BeautifulSoup(r_item.text, 'html.parser')

In [92]:
#funcion para iniciar el escraper de cada libro
def scraper_book(url):
    content_book={}
    r=requests.get(url)
    tire='Revista Mexicana de Ciencias Políticas y Sociales'
    a='V. Ciencias Sociales'
    tem='Sociología, Ciencia Política, Relaciones Internacionales, Administración Pública y Ciencias de la Comunicación'
    url='http://www.revistas.unam.mx/index.php/rmcpys/'
    s_item=BeautifulSoup(r.text,'html.parser')
    #titulo de revista
    try:
        titulo=tire
        content_book['Titulo de revista']=tire
    except AttributeError:
        content_book['Titulo de revista']=None
    #area
    try:
        area=a
        content_book['Área']=a
    except AttributeError:
        content_book['Área']=None
    #tematica
    try:
        tema=tem
        content_book['Temática']=tem
    except AttributeError:
        content_book['Temática']=None
    #titulo libro
    try:
        titu=s_item.find('h3', class_='mes_sub').get_text(strip=True)
        content_book['Titulo del libro']=titu
    except AttributeError:
        content_book['Titulo del libro']=None
    #resumen
    try:
        resu=s_item.find('a', class_='cover').get_text(strip=True)
        content_book['Resumen']=resu
    except AttributeError:
        content_book['Resumen']=None
    #Link incial
    try:
        inicial=url
        content_book['Link inicial']=url
    except AttributeError:
        content_book['Link inicial']=None
    #link articulo
    try:
        link=s_item.find('div', id='issueCoverImage').get('href')
        content_book['Link articulo']=urljoin(url_root, link)
    except AttributeError:
        content_book['Link articulo']=None
    return content_book

In [93]:
list_scraper=list_scraper[0:5]
datos_book=[]
for idx, i in enumerate(list_scraper):
    print(f'estas escrapeando la pag {idx}')
    datos_book.append(scraper_book(i))

estas escrapeando la pag 0
estas escrapeando la pag 1
estas escrapeando la pag 2
estas escrapeando la pag 3
estas escrapeando la pag 4


In [94]:
df_catalogo=pd.DataFrame(datos_book)
df_catalogo

,Titulo de revista,Área,Temática,Titulo del libro,Resumen,Link inicial,Link articulo
0,Revista Mexicana de Ciencias Políticas y Sociales,V. Ciencias Sociales,"Sociología, Ciencia Política, Relaciones Inter...",ESTRATEGIA Y CRISIS DE LOS EUA,None,http://www.revistas.unam.mx/index.php/rmcpys/,http://www.revistas.unam.mx/index.php/rmcpys/i...
1,Revista Mexicana de Ciencias Políticas y Sociales,V. Ciencias Sociales,"Sociología, Ciencia Política, Relaciones Inter...",FILOSOFÍA Y POLÍTICA,None,http://www.revistas.unam.mx/index.php/rmcpys/,http://www.revistas.unam.mx/index.php/rmcpys/i...
2,Revista Mexicana de Ciencias Políticas y Sociales,V. Ciencias Sociales,"Sociología, Ciencia Política, Relaciones Inter...",COMICS Y TELEVISIÓN,None,http://www.revistas.unam.mx/index.php/rmcpys/,http://www.revistas.unam.mx/index.php/rmcpys/i...
3,Revista Mexicana de Ciencias Políticas y Sociales,V. Ciencias Sociales,"Sociología, Ciencia Política, Relaciones Inter...",EL SISTEMA POLÍTICO MEXICANO,None,http://www.revistas.unam.mx/index.php/rmcpys/,http://www.revistas.unam.mx/index.php/rmcpys/i...
4,Revista Mexicana de Ciencias Políticas y Sociales,V. Ciencias Sociales,"Sociología, Ciencia Política, Relaciones Inter...",PSICOANÁLISIS Y SOCIEDAD,None,http://www.revistas.unam.mx/index.php/rmcpys/,http://www.revistas.unam.mx/index.php/rmcpys/i...


In [95]:
df_catalogo.to_csv('Revista49.07.csv', index=False)